In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_pzc_add_cl',
    proc_date,
    'temp_sap_pzc_add_cl'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_pzc_add_cl
(
  hash_id BIGINT,
  _bic_zc_add_cl STRING,
  objvers STRING,
  changed STRING,
  email_addr STRING,
  str_suppl3 STRING,
  str_suppl4 STRING,
  street60 STRING,
  name40 STRING,
  city_1 STRING,
  city_2 STRING,
  str_suppl2 STRING,
  oi_sort1 STRING,
  oi_sort2 STRING,
  proc_date DATE,
  file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
 """)

In [0]:
spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW sap_pzc_add_cl 
    AS
        WITH dat AS (
            SELECT 
                farm_fingerprint(CONCAT(
                    IFNULL(CAST(_bic_zc_add_cl AS STRING), ""), 
                    IFNULL(CAST(objvers AS STRING), "")
                )) hash_id,
                ROW_NUMBER() OVER (PARTITION BY _bic_zc_add_cl, objvers ORDER BY proc_date DESC) rn,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(_bic_zc_add_cl) AS _bic_zc_add_cl,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(objvers) AS objvers,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(changed) AS changed,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(email_addr) AS email_addr,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(str_suppl3) AS str_suppl3,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(str_suppl4) AS str_suppl4,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(street60) AS street60,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(name40) AS name40,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(city_1) AS city_1,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(city_2) AS city_2,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(str_suppl2) AS str_suppl2,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(oi_sort1) AS oi_sort1,
                {catalog_name}.default.fn_CLEAN_SAP_STRING(oi_sort2) AS oi_sort2,
                proc_date,
                file_creation_ts
            FROM temp_sap_pzc_add_cl
        )
        SELECT * EXCEPT (rn) FROM dat WHERE rn = 1
    """)

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_pzc_add_cl main_tbl
USING (
  SELECT * FROM sap_pzc_add_cl WHERE proc_date = cast('{proc_date}' as date)
) temp_tbl
ON main_tbl.hash_id = temp_tbl.hash_id

WHEN MATCHED THEN 
  UPDATE SET
    main_tbl.changed = temp_tbl.changed,
    main_tbl.email_addr = temp_tbl.email_addr,
    main_tbl.str_suppl3 = temp_tbl.str_suppl3,
    main_tbl.str_suppl4 = temp_tbl.str_suppl4,
    main_tbl.street60 = temp_tbl.street60,
    main_tbl.name40 = temp_tbl.name40,
    main_tbl.city_1 = temp_tbl.city_1,
    main_tbl.city_2 = temp_tbl.city_2,
    main_tbl.str_suppl2 = temp_tbl.str_suppl2,
    main_tbl.oi_sort1 = temp_tbl.oi_sort1,
    main_tbl.oi_sort2 = temp_tbl.oi_sort2

WHEN NOT MATCHED THEN 
  INSERT (
    hash_id,
    _bic_zc_add_cl,
    objvers,
    changed,
    email_addr,
    str_suppl3,
    str_suppl4,
    street60,
    name40,
    city_1,
    city_2,
    str_suppl2,
    oi_sort1,
    oi_sort2,
    proc_date,
    file_creation_ts
  )
  VALUES (
    temp_tbl.hash_id,
    temp_tbl._bic_zc_add_cl,
    temp_tbl.objvers,
    temp_tbl.changed,
    temp_tbl.email_addr,
    temp_tbl.str_suppl3,
    temp_tbl.str_suppl4,
    temp_tbl.street60,
    temp_tbl.name40,
    temp_tbl.city_1,
    temp_tbl.city_2,
    temp_tbl.str_suppl2,
    temp_tbl.oi_sort1,
    temp_tbl.oi_sort2,
    temp_tbl.proc_date,
    temp_tbl.file_creation_ts
  )
""")
